In [ ]:
from src.trust_region_sqp import TrustRegionSQPFilter
import numpy as np

In [ ]:
## Constants

constants = dict()
constants["gamma_0"] = 0.1
constants["gamma_1"] = 0.5
constants["gamma_2"] = 2.0
constants["eta_1"] = 0.01
constants["eta_2"] = 0.9
constants["mu"] = 0.01
constants["gamma_vartheta"] = 1E-4 
constants["kappa_vartheta"] = 1E-4
constants["kappa_radius"] = 0.7
constants["kappa_mu"] = 100
constants["kappa_tmd"] = 0.01

dataset = np.array([[0.0, 0.0], [1.0, 0.0], [0.0, 1.0], [-1.0, 0.0], [1.0, 1.0]]).T + np.array([[2.2], [2.2]])

def cf(x:np.ndarray) -> np.ndarray: # Rosenbrock function: OF
    return 1*(x[1]-x[0]**2)**2+((x[0]-1)**2)/100

def eq_constraint(x:np.ndarray) -> np.ndarray: # equality constraints
    return x[1] - x[0]

def ineq_constraint(x:np.ndarray) -> np.ndarray: # equality constraints
    return x[1] + x[0]

In [ ]:
tr = TrustRegionSQPFilter(constants = constants, 
                          dataset = dataset, 
                          cf = cf, 
                          eqcs = [eq_constraint], 
                          ineqcs = [ineq_constraint])
print(tr)